# 作业2
要求: 
    1. 尽量自己独立完成，如不懂可以问同学和老师，但是所有的代码都要自己输入，看懂，亲自运行
    2. 注意作业的截止时间，要在截止时间前提交
    3. 作业提交只需提交对应的ipynb后缀的文件
题目：

## 一、百度地图获取地理位置信息

参考书中4种获取地理位置信息的代码，原示例中的谷歌地图可改为百度地图，原示例中是输出地理位置信息的经度和纬度，将其改为可以输出指定地址的格式化地址信息。   
例如：  
hangzhou dianzi university的格式化地址信息：  
中国浙江省杭州市西湖区文三路电子信息街区文一路115号


In [28]:
import requests, json

# 百度API不能直接去找格式化地址信息，只能先找经纬度，再去查找该坐标对应的格式化地址信息
def geocode(access_key, addr):
    para = {
        'address': addr,
        'output': 'json',
        'ak': access_key
    }
    url = 'http://api.map.baidu.com/geocoding/v3/'
    res = requests.get(url, params=para)
    res = res.json()
    return res['result']['location']

def geoaddr(access_key, loc):
    para = {
        'ak': access_key,
        'output': 'json',
        'location': '{},{}'.format(loc['lat'], loc['lng'])
    }
    url = 'http://api.map.baidu.com/reverse_geocoding/v3/'
    res = requests.get(url, params=para)
    res = res.json()
    print(res['result']['formatted_address'])

if __name__ == '__main__':
    access_key = '*****'
    addr = '杭州电子科技大学'
    loc_code = geocode(access_key, addr)
    geoaddr(access_key, loc_code)


浙江省杭州市江干区金沙南路


如果谷歌地理信息服务实在无法访问，可以改成天气预报服务获取，参考课本中第2、3、4种方法来实现。  
参考下面的说明：
 https://www.sojson.com/api/weather.html   
 
例如：
输入：杭州   
输出格式为：
杭州：20180920，湿度95， 温度24， 空气质量优，各类人群可自由活动



In [61]:
import socket, json

# 直接通过城市名获取cityID的接口已经关闭，所以这里改成“输入cityID，输出该ID对应城市的天气预报
# 杭州ID：101210101
request_raw = '''GET /api/weather/city/{} HTTP/1.1\r
Host: t.weather.sojson.com\r
User-Agent: curl/7.58.0\r
Accept: */*\r
\r
'''
# 好像utf-8不是每次都能decode成功...跟网络状况有关系
def getweather(cityID):
    sock = socket.socket()
    sock.connect(('t.weather.sojson.com', 80))
    request_text = request_raw.format(cityID)
    sock.sendall(request_text.encode('ascii'))
    raw_reply = b''
    while True:
        tmp = sock.recv(1024)
        if not tmp:
            break
        raw_reply += tmp
    raw_text = raw_reply.decode()
    raw_text = raw_text.split('\r\n')
    data = json.loads(raw_text[len(raw_text)-4])
    out_s = '{}，{}，湿度{}，温度{}，空气质量{}，{}'.format(data['cityInfo']['city'], data['date'], data['data']['shidu'], data['data']['wendu'], data['data']['quality'], data['data']['ganmao'])
    print(out_s)

if __name__ == '__main__':
    cityID = int(input())
    getweather(cityID)


101210101
杭州市，20191011，湿度81%，温度21，空气质量轻度污染，儿童、老年人及心脏、呼吸系统疾病患者人群应减少长时间或高强度户外锻炼


## 二、掷骰子，猜大小游戏

### 第一步，完成骰点list的生成
下面是骰子的模版定义，分析其特征，利用str.split函数：
* 先用换行('\n')分隔符拆分为行，组成list
* 再用换行('，')分隔符拆分每个骰点(1~6)的部分
* 将每个骰点(1~6)的部分拼接，形成每个骰点的list

In [30]:
dice_tpl= '''\
┌───┐,┌───┐,┌───┐,┌───┐,┌───┐,┌───┐
│      │,│  ●  │,│●    │,│●  ●│,│●  ●│,│●  ●│
│  ●  │,│      │,│  ●  │,│      │,│  ●  │,│●  ●│
│      │,│  ●  │,│    ●│,│●  ●│,│●  ●│,│●  ●│
└───┘,└───┘,└───┘,└───┘,└───┘,└───┘'''
dice_lines = dice_tpl.split('\n')

for i in range(5):
    dice_lines[i] = dice_lines[i].split(',')

dice = ['','','','','','']

for i in range(6):
    for j in range(5):
        dice[i] += (dice_lines[j][i] + '\n')

for i in range(6):
    print(dice[i])


┌───┐
│      │
│  ●  │
│      │
└───┘

┌───┐
│  ●  │
│      │
│  ●  │
└───┘

┌───┐
│●    │
│  ●  │
│    ●│
└───┘

┌───┐
│●  ●│
│      │
│●  ●│
└───┘

┌───┐
│●  ●│
│  ●  │
│●  ●│
└───┘

┌───┐
│●  ●│
│●  ●│
│●  ●│
└───┘



代码应该输出：
```
┌───┐
│      │
│  ●  │
│      │
└───┘

┌───┐
│  ●  │
│      │
│  ●  │
└───┘

┌───┐
│●    │
│  ●  │
│    ●│
└───┘

┌───┐
│●  ●│
│      │
│●  ●│
└───┘

┌───┐
│●  ●│
│  ●  │
│●  ●│
└───┘

┌───┐
│●  ●│
│●  ●│
│●  ●│
└───┘
```

### 第二步，完成猜大小游戏
规则如下：    
　　投掷3个骰子，如果3个骰子之和小于10为小，大于10为大  
步骤分解： 
* 请用户输入大或小（用0,1代替）
* 投掷3个骰子，使用random库中的randint函数生成骰点大小，并依次输出骰点
* 计算骰点大小，判断用户输赢，并给出结果

**根据上面的步骤完成代码**

In [29]:
import random

user_input = bool(int(input()))
sum = 0

for i in range(3):
    tmp = random.randint(1, 6)
    sum += tmp
    print(dice[tmp-1])

if (sum > 10 and user_input is False) or (sum < 10 and user_input is True):
    print('输')
elif (sum > 10 and user_input is True) or (sum < 10 and user_input is False):
    print('赢')
else:
    print('平局')


1
┌───┐
│●    │
│  ●  │
│    ●│
└───┘

┌───┐
│●    │
│  ●  │
│    ●│
└───┘

┌───┐
│●  ●│
│●  ●│
│●  ●│
└───┘

赢
